In [2]:
# analyze-sgf를 사용해 SGF 파일을 분석하는 스크립트

import os
import subprocess

def process_sgf_files():
    # 나의 sgf 파일 디렉토리 경로 설정
    input_dir = 'my-sgf/'
    
    # analyze-sgf에 전달할 공통 옵션
    common_args = [
        'analyze-sgf', 
        '-a', 'maxVisits:5',  # 첫 번째 분석의 최대 방문 횟수
        '-g', 'minWinrateDropForVariations:8',  # 변형 분석을 위한 승률 변화 임계값
        '-r', '10'  # 재분석 시 최대 방문 횟수
    ]
    
    # input 디렉토리의 모든 .sgf 파일 순회
    for filename in os.listdir(input_dir):
        if filename.endswith('.sgf'):
            # 입력 파일의 전체 경로
            input_path = os.path.join(input_dir, filename)
            
            # 최종 명령어 구성 (common_args + 입력 파일 경로)
            full_command = common_args + [input_path]
            
            try:
                # subprocess를 사용해 analyze-sgf 실행
                result = subprocess.run(
                    full_command, 
                    capture_output=True, 
                    text=True
                )
                
                # 모듈의 stdout/stderr 출력
                if result.stdout:
                    print("Module stdout:", result.stdout)
                if result.stderr:
                    print("Module stderr:", result.stderr)
                
            except subprocess.CalledProcessError as e:
                # 모듈 실행 중 오류 발생 시
                print(f"Error processing {filename}: {e}")
            except Exception as e:
                # 기타 예외 처리
                print(f"Unexpected error with {filename}: {e}")

# 스크립트 실행
if __name__ == '__main__':
    process_sgf_files()

Module stdout: # Analyze-SGF Report

Komi 6.5, B+R, 2024-11-25

변상일 (Black)
* KataGo top choices (60.22%, 56/93)
* Less than 2% win rate drops (70.97%, 66/93)
* Less than 5% win rate drops (82.80%, 77/93)
* More than 5% win rate drops (17.20%, 16/93): #33 ⇣17.79%, #39 ⇣7.33%, #43 ⇣7.46%, #47 ⇣11.11%, #49 ⇣9.82%, #75 ⇣5.03%, #83 ⇣12.63%, #89 ⇣5.58%, #95 ⇣12.54%, #97 ⇣6.56%, #99 ⇣10.17%, #107 ⇣7.85%, #125 ⇣14.36%, #129 ⇣14.20%, #131 ⇣25.68%, #145 ⇣14.88%
* More than 20% win rate drops (1.08%, 1/93): #131 ⇣25.68%
* Top 10 win rate drops: #131 ⇣25.68%, #33 ⇣17.79%, #145 ⇣14.88%, #125 ⇣14.36%, #129 ⇣14.20%, #83 ⇣12.63%, #95 ⇣12.54%, #47 ⇣11.11%, #99 ⇣10.17%, #49 ⇣9.82%
* Top 10 score drops: #131 ⇣4.28, #145 ⇣4.23, #169 ⇣2.45, #167 ⇣2.20, #33 ⇣2.17, #83 ⇣1.66, #179 ⇣1.53, #125 ⇣1.52, #47 ⇣1.48, #95 ⇣1.44

辜梓豪 (White)
* KataGo top choices (51.09%, 47/92)
* Less than 2% win rate drops (68.48%, 63/92)
* Less than 5% win rate drops (78.26%, 72/92)
* More than 5% win rate drops (21.74%, 20/92): #

In [3]:
# 분석된 sgf의 승률을 저장하는 스크립트

import os
import re

# 폴더 경로 설정
input_folder = "my-sgf"
output_folder = "sgf-winrates"

def extract_win_rates(content):
    # 승률 정보 추출 패턴
    pattern = r'Move (\d+)\n\n\* Win rate: ([BW]) ([\d.]+)%'
    matches = re.finditer(pattern, content)
    results = []
    
    for match in matches:
        move_num = int(match.group(1))
        color = match.group(2)
        rate = float(match.group(3))
        results.append(f"Move {move_num:3d} - {'Black' if color == 'B' else 'White'}: {rate:.2f}%")
    
    return '\n'.join(results)

# 데이터 변환 함수
def convert_to_black_perspective(line):
    if "White:" in line:
        # 백 승률 추출
        white_pct = float(line.split("White:")[1].strip().replace("%", ""))
        # 흑 승률 계산 (100 - 백 승률)
        black_pct = round(100 - white_pct, 2)
        # 문자열 변환
        return line.split("White:")[0] + f"Black: {black_pct}%"
    return line

def process_sgf_file(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # 승률 데이터 추출
    win_rates = extract_win_rates(content)
    
    # 각 라인 변환
    converted_lines = [convert_to_black_perspective(line) for line in win_rates.split('\n')]
    converted_win_rates = '\n'.join(converted_lines)
    
    # 결과 저장
    with open(output_path, 'w', encoding='utf-8') as output_file:
        output_file.write(f"승률 변화:\n{converted_win_rates}")
    print(f"Processed: {output_path}")

# 출력 폴더가 없으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 모든 SGF 파일 처리
for filename in os.listdir(input_folder):
    if filename.endswith("-analyzed.sgf"):
        input_path = os.path.join(input_folder, filename)
        output_filename = filename.replace("-analyzed.sgf", "-winrates.txt")
        output_path = os.path.join(output_folder, output_filename)
        
        process_sgf_file(input_path, output_path)

Processed: sgf-winrates/guz-winrates.txt


In [24]:
import os
from langchain_community.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# OpenAI API 키 설정
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# 디렉토리 설정
INPUT_DIR = "sgf-winrates"
OUTPUT_DIR = "analysis-results"


def create_prompt():
    return PromptTemplate(
        input_variables=["moves", "highlights"],
        template="""

승률 데이터:
{moves}

주요 변화 지점:
{highlights}

요구사항: 
주어진 데이터는 바둑 경기 승률 row data입니다. 반드시 주어진 데이터를 기반으로 해설을 작성하고, 분석된 데이터 외의 정보는 포함하지 마세요. 데이터 해석에서 지나치게 모호한 표현이나 상상의 해설은 지양해주세요.
    
결과 형식:
<추천 하이라이트 구간>
    - 최소 3개 이상의 주요 승률 변화 구간을 선정하고, 각 구간의 승률 변화를 기술
    - 승률 변화는 W에서 B가 된 지점, B에서 W가 된 지점 등을 포함하여 구체적으로 작성
    - 설명은 간결하면서도 구체적으로 작성
    
어조:
    - "흑과 백 모두 이 구간에서 큰 승률 하락이 발생했습니다."
    - "백의 170 수에서 92.54% 하락과 흑의 171 수에서 92.57% 하락이 나타나 이 시점이 대국의 흐름을 크게 바꾸었습니다."

주의사항:
    - 모든 답변을 충분히 상세히 작성해주세요.
    - 분석 결과는 반드시 데이터를 기반으로 하며, 명확하고 직관적인 흐름으로 작성합니다.
    - 형식을 반드시 준수하며, 내용의 순서를 어기지 않도록 주의하세요.
    - 해설 외의 내용은 포함하지 않도록 주의해주세요.

참고 정보:
    - B+R은 흑 승리, W+R은 백 승리를 의미합니다.
    - "흑 우세"는 흑의 유리함. "백 우세"는 백의 유리함을 의미합니다.
    
출력 예시:
    추천 하이라이트 지점
	1. #168 ~ #172 수:
        - 흑과 백 모두 이 구간에서 큰 승률 하락이 발생했습니다.
        - 백의 #168, #170, #172 수와 흑의 #169, #171 수에서 각각 큰 승률 하락이 있었으며, 백의 #170 수에서 92.54% 하락과 흑의 #171 수에서 92.57% 하락이 나타나 이 시점이 대국의 흐름을 크게 바꾸었습니다.
        - 이 구간은 대국의 주도권이 크게 바뀌는 핵심적인 중반 구간입니다.
	2. #193 ~ #196 수:
        - 흑의 #193, #195 수와 백의 #194, #196 수에서 각각 60% 이상 승률이 떨어졌습니다.
        - 이는 양측의 큰 실수가 교차된 또 하나의 중요한 시점으로, 흑과 백의 주요 전략적 오류가 겹쳐지며 승부의 흐름이 요동친 것으로 보입니다.
	3. #247 ~ #249 수:
        - 흑의 #249 수는 98.87% 승률 하락으로, 대국을 사실상 결정지은 수 중 하나일 가능성이 높습니다.
        - 이 지점은 중후반부에서의 결정적인 순간으로, 흑이 치명적 실수를 범해 백에게 승리를 내준 수일 수 있습니다.
    ...(더 있으면 추가)

"""
    )

def analyze_game(input_path):
    with open(input_path, 'r', encoding='utf-8') as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]
    
    moves = []
    highlights = []
    prev_rate = None
    
    for idx, line in enumerate(lines):
        if ' - ' not in line:
            continue
            
        move_part, rate_part = line.split(' - ')
        move_num = int(move_part.replace('Move', '').strip())
        rate = float(rate_part.split(':')[1].replace('%', '').strip())
        
        moves.append(f"수순 {move_num}: 흑 {rate:.2f}%")
        
        if prev_rate is not None:
            change = rate - prev_rate
            
            # 1. 50% 기준점 교차 감지
            if (prev_rate < 50 and rate >= 50) or (prev_rate > 50 and rate <= 50):
                highlights.append({
                    "id": len(highlights) + 1,
                    "range": f"{move_num-1}-{move_num}",
                    "description": f"수순 {move_num}에서 승패 분기점 통과: {prev_rate:.2f}% → {rate:.2f}%"
                })
            
            # 2. 8% 이상 변화 감지
            elif abs(change) >= 8.0:
                highlights.append({
                    "id": len(highlights) + 1,
                    "range": f"{move_num-1}-{move_num}",
                    "description": f"수순 {move_num}에서 큰 변화: {abs(change):.2f}% ({'흑 개선' if change > 0 else '백 개선'})"
                })
        
        prev_rate = rate
    
    llm = ChatOpenAI(
        temperature=0.7, 
        model_name="gpt-4o",
        openai_api_key=OPENAI_API_KEY
    )
    chain = LLMChain(llm=llm, prompt=create_prompt())
    
    analysis = chain.run(
        moves="\n".join(moves),
        highlights=json.dumps(highlights, ensure_ascii=False, indent=2)
    )
    
    return analysis, highlights

def main():
    for filename in os.listdir(INPUT_DIR):
        if filename.endswith("-winrates.txt"):
            input_path = os.path.join(INPUT_DIR, filename)
            base_name = filename.replace("-winrates.txt", "")
            
            print(f"분석 중: {filename}")
            analysis, highlights = analyze_game(input_path)
            
            analysis_path = os.path.join(OUTPUT_DIR, f"{base_name}-analysis.txt")
            with open(analysis_path, 'w', encoding='utf-8') as f:
                f.write(analysis)
            
            highlights_path = os.path.join(OUTPUT_DIR, f"{base_name}-highlights.json")
            with open(highlights_path, 'w', encoding='utf-8') as f:
                json.dump(highlights, f, ensure_ascii=False, indent=2)
            
            print(f"완료: {base_name}")

if __name__ == "__main__":
    main()

분석 중: guz-winrates.txt
완료: guz
